In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('data.pkl')

In [3]:
df = df[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    'zip_code',
    'item_category_id',
    'category_type',
    'category_device',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'delta_price_trend_with_lag_1',
    'month',
    'days_in_month',
    'item_last_sold_in_given_shop',
    'item_last_sold',
    'item_first_sold_in_given_shop',
    'item_first_sold'
]]

In [4]:
X_train = df[df.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = df[df.date_block_num < 33]['item_cnt_month']
X_valid = df[df.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = df[df.date_block_num == 33]['item_cnt_month']
X_test = df[df.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [5]:
from xgboost import XGBRegressor

model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300,
    colsample_bytree=0.8,
    subsample=0.8,
    eta=0.3,
    seed=42)

model.fit(
    X_train,
    Y_train,
    eval_metric="rmse",
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
    verbose=True,
    early_stopping_rounds = 10)

test = pd.read_csv('data/technical/test.csv')

Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index,
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)

[0]	validation_0-rmse:1.05972	validation_1-rmse:1.03928
[1]	validation_0-rmse:0.96190	validation_1-rmse:0.95841
[2]	validation_0-rmse:0.90170	validation_1-rmse:0.90892
[3]	validation_0-rmse:0.86658	validation_1-rmse:0.90337
[4]	validation_0-rmse:0.84649	validation_1-rmse:0.89424
[5]	validation_0-rmse:0.83445	validation_1-rmse:0.88339
[6]	validation_0-rmse:0.82173	validation_1-rmse:0.89428
[7]	validation_0-rmse:0.81196	validation_1-rmse:0.89773
[8]	validation_0-rmse:0.80584	validation_1-rmse:0.90558
[9]	validation_0-rmse:0.80223	validation_1-rmse:0.90521
[10]	validation_0-rmse:0.79623	validation_1-rmse:0.91530
[11]	validation_0-rmse:0.79232	validation_1-rmse:0.91954
[12]	validation_0-rmse:0.79005	validation_1-rmse:0.92130
[13]	validation_0-rmse:0.78773	validation_1-rmse:0.91388
[14]	validation_0-rmse:0.78490	validation_1-rmse:0.90675
